In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

# Load the test dataset
df = pd.read_csv('/content/test_r3.csv')  # Make sure to update the path to your dataset location


In [2]:
!pip install peft

In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM

config = PeftConfig.from_pretrained("lorahub/flan_t5_large-anli_r3")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
model = PeftModel.from_pretrained(model, "lorahub/flan_t5_large-anli_r3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
!pip install tqdm


In [5]:
# Define the custom dataset
class ANLIDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item = self.dataframe.iloc[idx]
        premise = item['premise']
        hypothesis = item['hypothesis']
        reason = item['reason']
        label = self.label_map[item['label']]
        # Format the input text in a way that's expected by the T5 model
        input_sequence = f"premise: {premise} hypothesis: {hypothesis} reason: {reason}"
        return input_sequence, label


In [6]:
dataset = ANLIDataset(df)

# DataLoader setup
loader = DataLoader(dataset, batch_size=32, shuffle=False)


# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 1024)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
       

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")


In [8]:
from tqdm import tqdm
# Evaluation loop with tqdm
predictions = []
labels = []

for batch in tqdm(loader, desc="Evaluating"):
    input_sequences, batch_labels = batch
    # Tokenize the inputs. Adjust as per your tokenizer's requirement
    inputs = tokenizer(input_sequences, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move tensors to the same device as model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        # Forward pass through the model, specifying max_new_tokens
        output_sequences = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_new_tokens=50  # Adjust as needed
        )
        # Decode the generated sequences to text
        pred_texts = [tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for generated_id in output_sequences]
    predictions.extend(pred_texts)
    labels.extend(batch_labels)



Evaluating: 100%|██████████| 38/38 [01:24<00:00,  2.23s/it]


In [9]:
def map_prediction_to_label(prediction):
    mapping = {
        "yes": "entailment",
        "no": "contradiction",
        "it's impossible to say": "neutral"
    }
    return mapping.get(prediction.strip().lower(), "unknown")

mapped_predictions = [map_prediction_to_label(pred) for pred in predictions]

# Recalculate accuracy with the mapped predictions
correct_predictions = sum(1 for mapped_pred, label in zip(mapped_predictions, labels) if mapped_pred == label.strip().lower())
accuracy = correct_predictions / len(labels)
print(f"Corrected Accuracy: {accuracy:.2f}")


Corrected Accuracy: 0.65
